# Setting up the notebook
- importing the necessart packages to help explore the data
- loading the data 
- as well creating a sparks session

In [0]:
# importing the packaging 
sqlContext 
from pyspark.sql.functions import when, col
from pyspark.sql.functions import year, to_date
from pyspark.sql.functions import trim
from pyspark.sql import functions as F
from pyspark.sql.functions import substring
from pyspark.sql.types import IntegerType,BooleanType,DateType
from pyspark.sql.functions import to_timestamp,sum,year
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, format_number, regexp_replace
from pyspark.sql.functions import desc,asc
from pyspark.sql.functions import count

In [0]:
# create a SparkSession
spark = SparkSession.builder.getOrCreate()

In [0]:
# loading the csvs into the notebook 
resturant_1_price = sqlContext.read.load('/FileStore/tables/restaurant_1_products_price.csv', format='csv',header = True,inferSchema=True)
resturant_2_price = sqlContext.read.load('/FileStore/tables/restaurant_2_products_price.csv', format='csv',header = True,inferSchema=True)
resturant_1_order = sqlContext.read.load('/FileStore/tables/restaurant_1_orders.csv', format='csv',header = True,inferSchema=True)
resturant_2_order = sqlContext.read.load('/FileStore/tables/restaurant_2_orders.csv', format='csv',header = True,inferSchema=True)


# Introduction

In this notebook will be focusing on the on the indian takeaway, through the use of SQL spark, to manuiplate data before the analyses. After doing the data prep, will then focus on the data exploration by formulating 4 questions to help focus on the data explorations and find any useful insights that can I get based on the analyses.

4 Questions that will be exploring and answering:

Question 1: Which resturant has the most sales based on the numnber of orders and by how much?

Question 2: Which resturants produced the highest profits for during the course of 5 years?

Question 3: What is the the most expensive products from the two resturants? What is their least expensive product?

Question 4: Which customer from the two resturants placed the most order, for all of the years?

By the ending of this notebook, will developed a further understanding of the two resturants are and as well having an insights which resturant is doiong better in terms of profits or any other factors that has been discovered.

# Basic Data Prep

## Resturant 1

In [0]:
# displaying the table for all the product prices in resturant 1 
resturant_1_price.show()

+-------------+-------------+
|    Item Name|Product Price|
+-------------+-------------+
|   Mint Sauce|          0.5|
|  Lime Pickle|          0.5|
|Mango Chutney|          0.5|
|    Red Sauce|          0.5|
|Onion Chutney|          0.5|
|Plain Papadum|          0.8|
|Spicy Papadum|          0.8|
|      Chapati|         1.95|
|        Puree|         1.95|
|Tandoori Roti|          2.6|
|   Plain Naan|          2.6|
|       Raitha|         2.95|
|   Keema Naan|         2.95|
|Peshwari Naan|         2.95|
|   Pilau Rice|         2.95|
|  Green Salad|         2.95|
|  Bottle Coke|         2.95|
|   Plain Rice|         2.95|
|  Garlic Naan|         2.95|
|   Onion Naan|         2.95|
+-------------+-------------+
only showing top 20 rows



In [0]:
# checking the objects type
resturant_1_price.dtypes

Out[233]: [('Item Name', 'string'), ('Product Price', 'double')]

In [0]:
# check for any missing values fror product price
missing_values = resturant_1_price.agg(*[sum(col(c).isNull().cast("integer")).alias(c) for c in resturant_1_price.columns])

missing_values.show()

+---------+-------------+
|Item Name|Product Price|
+---------+-------------+
|        0|            0|
+---------+-------------+



In [0]:
# viewing the resturant order
resturant_1_order.show()

+------------+----------------+--------------------+--------+-------------+--------------+
|Order Number|      Order Date|           Item Name|Quantity|Product Price|Total products|
+------------+----------------+--------------------+--------+-------------+--------------+
|       16118|03/08/2019 20:25|       Plain Papadum|       2|          0.8|             6|
|       16118|03/08/2019 20:25|    King Prawn Balti|       1|        12.95|             6|
|       16118|03/08/2019 20:25|         Garlic Naan|       1|         2.95|             6|
|       16118|03/08/2019 20:25|       Mushroom Rice|       1|         3.95|             6|
|       16118|03/08/2019 20:25| Paneer Tikka Masala|       1|         8.95|             6|
|       16118|03/08/2019 20:25|       Mango Chutney|       1|          0.5|             6|
|       16117|03/08/2019 20:17|          Plain Naan|       1|          2.6|             7|
|       16117|03/08/2019 20:17|       Mushroom Rice|       1|         3.95|             7|

In [0]:
# converting the Order Number into datetime 
resturant_1_order = resturant_1_order.withColumn('Order Date', to_date(resturant_1_order['Order Date'], 'MM/dd/yyyy HH:mm'))

# display the updated DataFrame
resturant_1_order.show()

+------------+----------+--------------------+--------+-------------+--------------+
|Order Number|Order Date|           Item Name|Quantity|Product Price|Total products|
+------------+----------+--------------------+--------+-------------+--------------+
|       16118|2019-03-08|       Plain Papadum|       2|          0.8|             6|
|       16118|2019-03-08|    King Prawn Balti|       1|        12.95|             6|
|       16118|2019-03-08|         Garlic Naan|       1|         2.95|             6|
|       16118|2019-03-08|       Mushroom Rice|       1|         3.95|             6|
|       16118|2019-03-08| Paneer Tikka Masala|       1|         8.95|             6|
|       16118|2019-03-08|       Mango Chutney|       1|          0.5|             6|
|       16117|2019-03-08|          Plain Naan|       1|          2.6|             7|
|       16117|2019-03-08|       Mushroom Rice|       1|         3.95|             7|
|       16117|2019-03-08|Tandoori Chicken ...|       1|         4

In [0]:
# extracting the year values from the Order Date columns for resturant 1 orders df
resturant_1_order = resturant_1_order.withColumn('Year', year(resturant_1_order['Order Date']))
resturant_1_order.show()

+------------+----------+--------------------+--------+-------------+--------------+----+
|Order Number|Order Date|           Item Name|Quantity|Product Price|Total products|Year|
+------------+----------+--------------------+--------+-------------+--------------+----+
|       16118|2019-03-08|       Plain Papadum|       2|          0.8|             6|2019|
|       16118|2019-03-08|    King Prawn Balti|       1|        12.95|             6|2019|
|       16118|2019-03-08|         Garlic Naan|       1|         2.95|             6|2019|
|       16118|2019-03-08|       Mushroom Rice|       1|         3.95|             6|2019|
|       16118|2019-03-08| Paneer Tikka Masala|       1|         8.95|             6|2019|
|       16118|2019-03-08|       Mango Chutney|       1|          0.5|             6|2019|
|       16117|2019-03-08|          Plain Naan|       1|          2.6|             7|2019|
|       16117|2019-03-08|       Mushroom Rice|       1|         3.95|             7|2019|
|       16

In [0]:
# checking the datatypes again
resturant_1_order.dtypes

Out[238]: [('Order Number', 'int'),
 ('Order Date', 'date'),
 ('Item Name', 'string'),
 ('Quantity', 'int'),
 ('Product Price', 'double'),
 ('Total products', 'int'),
 ('Year', 'int')]

In [0]:
# checking the missing values
missing_values = resturant_1_order.agg(*[sum(col(c).isNull().cast("integer")).alias(c) for c in resturant_1_order.columns])

missing_values.show()

+------------+----------+---------+--------+-------------+--------------+-----+
|Order Number|Order Date|Item Name|Quantity|Product Price|Total products| Year|
+------------+----------+---------+--------+-------------+--------------+-----+
|           0|     46162|        0|       0|            0|             0|46162|
+------------+----------+---------+--------+-------------+--------------+-----+



In [0]:
# adding 0 for no years where it contains the null values 
resturant_1_order = resturant_1_order.fillna(0)
resturant_1_order.show()

+------------+----------+--------------------+--------+-------------+--------------+----+
|Order Number|Order Date|           Item Name|Quantity|Product Price|Total products|Year|
+------------+----------+--------------------+--------+-------------+--------------+----+
|       16118|2019-03-08|       Plain Papadum|       2|          0.8|             6|2019|
|       16118|2019-03-08|    King Prawn Balti|       1|        12.95|             6|2019|
|       16118|2019-03-08|         Garlic Naan|       1|         2.95|             6|2019|
|       16118|2019-03-08|       Mushroom Rice|       1|         3.95|             6|2019|
|       16118|2019-03-08| Paneer Tikka Masala|       1|         8.95|             6|2019|
|       16118|2019-03-08|       Mango Chutney|       1|          0.5|             6|2019|
|       16117|2019-03-08|          Plain Naan|       1|          2.6|             7|2019|
|       16117|2019-03-08|       Mushroom Rice|       1|         3.95|             7|2019|
|       16

In [0]:
# checking the missing values for years to check the null value has been removed 
missing_values = resturant_1_order.agg(*[sum(col(c).isNull().cast("integer")).alias(c) for c in resturant_1_order.columns])
missing_values.show()

+------------+----------+---------+--------+-------------+--------------+----+
|Order Number|Order Date|Item Name|Quantity|Product Price|Total products|Year|
+------------+----------+---------+--------+-------------+--------------+----+
|           0|     46162|        0|       0|            0|             0|   0|
+------------+----------+---------+--------+-------------+--------------+----+



## Resturant 2

In [0]:
# displaying the table for all the product prices in resturant 1 
resturant_2_price.show()

+-------------+-------------+
|    Item Name|Product Price|
+-------------+-------------+
|Onion Chutney|          0.5|
|   Mint Sauce|          0.5|
|Mango Chutney|          0.5|
|    Red Sauce|          0.5|
|  Lime Pickle|          0.5|
|Plain Papadum|          0.8|
|Spicy Papadum|          0.8|
|      Chapati|         1.95|
|        Puree|         1.95|
|         Dahi|         1.95|
|         Naan|          2.5|
|Tandoori Roti|          2.5|
|   Keema Naan|         2.95|
|   Pilau Rice|         2.95|
|  Garlic Naan|         2.95|
|   Plain Rice|         2.95|
|  Bottle Coke|         2.95|
|Peshwari Naan|         2.95|
|      Paratha|         2.95|
|  Green Salad|         2.95|
+-------------+-------------+
only showing top 20 rows



In [0]:
# checking the objects type
resturant_2_price.dtypes

Out[243]: [('Item Name', 'string'), ('Product Price', 'double')]

In [0]:
# checking the missing values
missing_values = resturant_2_price.agg(*[sum(col(c).isNull().cast("integer")).alias(c) for c in resturant_2_price.columns])
missing_values.show()

+---------+-------------+
|Item Name|Product Price|
+---------+-------------+
|        0|            0|
+---------+-------------+



In [0]:
# displaying the table for all the orders in resturant 2
resturant_2_order.show()

+--------+----------------+--------------------+--------+-------------+--------------+
|Order ID|      Order Date|           Item Name|Quantity|Product Price|Total products|
+--------+----------------+--------------------+--------+-------------+--------------+
|   25583|03/08/2019 21:58|Tandoori Mixed Grill|       1|        11.95|            12|
|   25583|03/08/2019 21:58|        Madras Sauce|       1|         3.95|            12|
|   25583|03/08/2019 21:58|       Mushroom Rice|       2|         3.95|            12|
|   25583|03/08/2019 21:58|         Garlic Naan|       1|         2.95|            12|
|   25583|03/08/2019 21:58|             Paratha|       1|         2.95|            12|
|   25583|03/08/2019 21:58|          Plain Rice|       1|         2.95|            12|
|   25583|03/08/2019 21:58|         Prawn Puree|       1|         4.95|            12|
|   25583|03/08/2019 21:58|       Plain Papadum|       1|          0.8|            12|
|   25583|03/08/2019 21:58|       Mango Chu

In [0]:
# checking the datatypes
resturant_2_order.dtypes

Out[246]: [('Order ID', 'int'),
 ('Order Date', 'string'),
 ('Item Name', 'string'),
 ('Quantity', 'int'),
 ('Product Price', 'double'),
 ('Total products', 'int')]

In [0]:
# converting the order date to date time as it was string 
resturant_2_order = resturant_2_order.withColumn('Order Date', to_date(resturant_2_order['Order Date'], 'MM/dd/yyyy HH:mm'))

# dsisplay the updated DataFrame
resturant_2_order.show()

+--------+----------+--------------------+--------+-------------+--------------+
|Order ID|Order Date|           Item Name|Quantity|Product Price|Total products|
+--------+----------+--------------------+--------+-------------+--------------+
|   25583|2019-03-08|Tandoori Mixed Grill|       1|        11.95|            12|
|   25583|2019-03-08|        Madras Sauce|       1|         3.95|            12|
|   25583|2019-03-08|       Mushroom Rice|       2|         3.95|            12|
|   25583|2019-03-08|         Garlic Naan|       1|         2.95|            12|
|   25583|2019-03-08|             Paratha|       1|         2.95|            12|
|   25583|2019-03-08|          Plain Rice|       1|         2.95|            12|
|   25583|2019-03-08|         Prawn Puree|       1|         4.95|            12|
|   25583|2019-03-08|       Plain Papadum|       1|          0.8|            12|
|   25583|2019-03-08|       Mango Chutney|       2|          0.5|            12|
|   25583|2019-03-08|       

In [0]:
# extracting the year values from the Order Date columns for resturant 2 orders df
resturant_2_order = resturant_2_order.withColumn('Year', year(resturant_2_order['Order Date']))
resturant_2_order.show()

+--------+----------+--------------------+--------+-------------+--------------+----+
|Order ID|Order Date|           Item Name|Quantity|Product Price|Total products|Year|
+--------+----------+--------------------+--------+-------------+--------------+----+
|   25583|2019-03-08|Tandoori Mixed Grill|       1|        11.95|            12|2019|
|   25583|2019-03-08|        Madras Sauce|       1|         3.95|            12|2019|
|   25583|2019-03-08|       Mushroom Rice|       2|         3.95|            12|2019|
|   25583|2019-03-08|         Garlic Naan|       1|         2.95|            12|2019|
|   25583|2019-03-08|             Paratha|       1|         2.95|            12|2019|
|   25583|2019-03-08|          Plain Rice|       1|         2.95|            12|2019|
|   25583|2019-03-08|         Prawn Puree|       1|         4.95|            12|2019|
|   25583|2019-03-08|       Plain Papadum|       1|          0.8|            12|2019|
|   25583|2019-03-08|       Mango Chutney|       2|   

In [0]:
# checking the datatypes again
resturant_2_order.dtypes

Out[249]: [('Order ID', 'int'),
 ('Order Date', 'date'),
 ('Item Name', 'string'),
 ('Quantity', 'int'),
 ('Product Price', 'double'),
 ('Total products', 'int'),
 ('Year', 'int')]

In [0]:
# checking the missing values
missing_values = resturant_2_order.agg(*[sum(col(c).isNull().cast("integer")).alias(c) for c in resturant_2_order.columns])
missing_values.show()

+--------+----------+---------+--------+-------------+--------------+-----+
|Order ID|Order Date|Item Name|Quantity|Product Price|Total products| Year|
+--------+----------+---------+--------+-------------+--------------+-----+
|       0|     72574|        0|       0|            0|             0|72574|
+--------+----------+---------+--------+-------------+--------------+-----+



In [0]:
# adding 0 for no years where it contains the null values 
resturant_2_order = resturant_2_order.fillna(0)
resturant_1_order.show()

+------------+----------+--------------------+--------+-------------+--------------+----+
|Order Number|Order Date|           Item Name|Quantity|Product Price|Total products|Year|
+------------+----------+--------------------+--------+-------------+--------------+----+
|       16118|2019-03-08|       Plain Papadum|       2|          0.8|             6|2019|
|       16118|2019-03-08|    King Prawn Balti|       1|        12.95|             6|2019|
|       16118|2019-03-08|         Garlic Naan|       1|         2.95|             6|2019|
|       16118|2019-03-08|       Mushroom Rice|       1|         3.95|             6|2019|
|       16118|2019-03-08| Paneer Tikka Masala|       1|         8.95|             6|2019|
|       16118|2019-03-08|       Mango Chutney|       1|          0.5|             6|2019|
|       16117|2019-03-08|          Plain Naan|       1|          2.6|             7|2019|
|       16117|2019-03-08|       Mushroom Rice|       1|         3.95|             7|2019|
|       16

In [0]:
# checking the missing values if the null values for years has been inplace
missing_values = resturant_2_order.agg(*[sum(col(c).isNull().cast("integer")).alias(c) for c in resturant_2_order.columns])
missing_values.show()

+--------+----------+---------+--------+-------------+--------------+----+
|Order ID|Order Date|Item Name|Quantity|Product Price|Total products|Year|
+--------+----------+---------+--------+-------------+--------------+----+
|       0|     72574|        0|       0|            0|             0|   0|
+--------+----------+---------+--------+-------------+--------------+----+



# Data Exploration

## Question 1: Which resturant has the most sales based on the numnber of orders and by how much?

### Resturant 1

In [0]:
# Get the distinct values of years - choose any dataframes since the year values will be the same
distinct_years = resturant_2_order.select('Year').distinct().show()

+----+
|Year|
+----+
|2018|
|2019|
|2016|
|2017|
|   0|
+----+



In [0]:
# select specific columns using the chain method
df_year_1 = resturant_1_order.select("Item Name","Total Products", "Year")
df_year_1.show()

+--------------------+--------------+----+
|           Item Name|Total Products|Year|
+--------------------+--------------+----+
|       Plain Papadum|             6|2019|
|    King Prawn Balti|             6|2019|
|         Garlic Naan|             6|2019|
|       Mushroom Rice|             6|2019|
| Paneer Tikka Masala|             6|2019|
|       Mango Chutney|             6|2019|
|          Plain Naan|             7|2019|
|       Mushroom Rice|             7|2019|
|Tandoori Chicken ...|             7|2019|
|     Vindaloo - Lamb|             7|2019|
|             Chapati|             7|2019|
|          Lamb Tikka|             7|2019|
|         Saag Paneer|             7|2019|
|          Aloo Chaat|             5|2019|
|      Chicken Pakora|             5|2019|
|Chicken Chilli Ga...|             5|2019|
|        Lamb Biryani|             5|2019|
|          Keema Naan|             5|2019|
|      Chicken Pakora|             7|2019|
|       Plain Papadum|             7|2019|
+----------

In [0]:
# group by "Year" column and get total counts (resturant 1) and as well getting the total sum
# counting the number of products each year 
df_grouped_year1 = df_year_1.groupBy('Year').agg(count('*').alias('Resturant_1_Count'))

# sort the data by "Year" column in ascending order
df_grouped_year1_sorted = df_grouped_year1.orderBy('Year')
df_grouped_year1_sorted.show()

+----+-----------------+
|Year|Resturant_1_Count|
+----+-----------------+
|   0|            46162|
|2015|               49|
|2016|             3910|
|2017|             7846|
|2018|             9699|
|2019|             7152|
+----+-----------------+



### Resturant 2

In [0]:
# select specific columns using the chain method
df_year_2 = resturant_2_order.select("Item Name","Total Products", "Year")
df_year_2.show()

+--------------------+--------------+----+
|           Item Name|Total Products|Year|
+--------------------+--------------+----+
|Tandoori Mixed Grill|            12|2019|
|        Madras Sauce|            12|2019|
|       Mushroom Rice|            12|2019|
|         Garlic Naan|            12|2019|
|             Paratha|            12|2019|
|          Plain Rice|            12|2019|
|         Prawn Puree|            12|2019|
|       Plain Papadum|            12|2019|
|       Mango Chutney|            12|2019|
|       Onion Chutney|            12|2019|
|          Mint Sauce|            12|2019|
|Chicken Tikka Masala|            12|2019|
|Tandoori King Pra...|             5|2019|
|          Pilau Rice|             5|2019|
|       Peshwari Naan|             5|2019|
|         Bottle Coke|             5|2019|
|      Butter Chicken|             5|2019|
|         Saag Paneer|             5|2019|
|             Chapati|             5|2019|
|         Onion Bhaji|             5|2019|
+----------

In [0]:
# group by "Year" column and get total counts (resturant 1) and as well getting the total sum
# compute the total sum of counts
df_grouped_year2 = df_year_2.groupBy('Year').agg(count('*').alias('Resturant_2_Count'))

# sort the data by "Year" column in ascending order
df_grouped_year2_sorted = df_grouped_year2.orderBy('Year')
df_grouped_year2_sorted.show()

+----+-----------------+
|Year|Resturant_2_Count|
+----+-----------------+
|   0|            72574|
|2016|             5878|
|2017|            14057|
|2018|            15901|
|2019|            10773|
+----+-----------------+



### Combining the two resturant together

In [0]:
# combining the two grouped resturant for comparison
joined_year_data = df_grouped_year2_sorted.join(df_grouped_year1_sorted, on='Year',how='outer')
joined_year_data.show()

+----+-----------------+-----------------+
|Year|Resturant_2_Count|Resturant_1_Count|
+----+-----------------+-----------------+
|   0|            72574|            46162|
|2015|             null|               49|
|2016|             5878|             3910|
|2017|            14057|             7846|
|2018|            15901|             9699|
|2019|            10773|             7152|
+----+-----------------+-----------------+



In [0]:
# removing the null values and adding 0 in place of it
# fill null values in specified columns with 0
joined_year_data = joined_year_data.withColumn("Resturant_2_Count", when(col("Resturant_2_Count").isNull(), 0).otherwise(col("Resturant_2_Count")))

# show the DataFrame with null values replaced by 0
joined_year_data.show()

+----+-----------------+-----------------+
|Year|Resturant_2_Count|Resturant_1_Count|
+----+-----------------+-----------------+
|   0|            72574|            46162|
|2015|                0|               49|
|2016|             5878|             3910|
|2017|            14057|             7846|
|2018|            15901|             9699|
|2019|            10773|             7152|
+----+-----------------+-----------------+



In [0]:
# subtracting the difference so we can which company is selling better 
df_with_subtraction = joined_year_data.withColumn("Subtraction", col("Resturant_2_Count") - col("Resturant_1_Count"))
df_with_subtraction.show()

+----+-----------------+-----------------+-----------+
|Year|Resturant_2_Count|Resturant_1_Count|Subtraction|
+----+-----------------+-----------------+-----------+
|   0|            72574|            46162|      26412|
|2015|                0|               49|        -49|
|2016|             5878|             3910|       1968|
|2017|            14057|             7846|       6211|
|2018|            15901|             9699|       6202|
|2019|            10773|             7152|       3621|
+----+-----------------+-----------------+-----------+



### Answer
The most sales which the resturant has is Resturant 2, as resturant 2 shows significantly higher sales in product for example for year 2016 it shows that resturant 2 sold 1968 more products the resturant 1 which is in the postive numbers and so forth with the following year. However there is -49 between the resturanr_2 and resturant_2 count and the possible reason the resturant 2 has not opened in 2015.

## Question 2: Which resturants produced the highest profits for during the course of 5 years?

### Resturant 1

In [0]:
# view resturant 1 
resturant_1_order.show()

+------------+----------+--------------------+--------+-------------+--------------+----+
|Order Number|Order Date|           Item Name|Quantity|Product Price|Total products|Year|
+------------+----------+--------------------+--------+-------------+--------------+----+
|       16118|2019-03-08|       Plain Papadum|       2|          0.8|             6|2019|
|       16118|2019-03-08|    King Prawn Balti|       1|        12.95|             6|2019|
|       16118|2019-03-08|         Garlic Naan|       1|         2.95|             6|2019|
|       16118|2019-03-08|       Mushroom Rice|       1|         3.95|             6|2019|
|       16118|2019-03-08| Paneer Tikka Masala|       1|         8.95|             6|2019|
|       16118|2019-03-08|       Mango Chutney|       1|          0.5|             6|2019|
|       16117|2019-03-08|          Plain Naan|       1|          2.6|             7|2019|
|       16117|2019-03-08|       Mushroom Rice|       1|         3.95|             7|2019|
|       16

In [0]:
# creating the profit columns by multiplying the product price with quantity
df_new_col1 = resturant_1_order.withColumn("Profits_Resturant1", col("Product Price") * col("Quantity"))
df_new_col1.show()

+------------+----------+--------------------+--------+-------------+--------------+----+------------------+
|Order Number|Order Date|           Item Name|Quantity|Product Price|Total products|Year|Profits_Resturant1|
+------------+----------+--------------------+--------+-------------+--------------+----+------------------+
|       16118|2019-03-08|       Plain Papadum|       2|          0.8|             6|2019|               1.6|
|       16118|2019-03-08|    King Prawn Balti|       1|        12.95|             6|2019|             12.95|
|       16118|2019-03-08|         Garlic Naan|       1|         2.95|             6|2019|              2.95|
|       16118|2019-03-08|       Mushroom Rice|       1|         3.95|             6|2019|              3.95|
|       16118|2019-03-08| Paneer Tikka Masala|       1|         8.95|             6|2019|              8.95|
|       16118|2019-03-08|       Mango Chutney|       1|          0.5|             6|2019|               0.5|
|       16117|2019-

In [0]:
# creating a subdataframe by selcting which columns we want from resturant 1
df_profits_1 = df_new_col1.select("Year","Profits_Resturant1")
df_profits_1.show()

+----+------------------+
|Year|Profits_Resturant1|
+----+------------------+
|2019|               1.6|
|2019|             12.95|
|2019|              2.95|
|2019|              3.95|
|2019|              8.95|
|2019|               0.5|
|2019|               2.6|
|2019|              3.95|
|2019|              4.95|
|2019|              7.95|
|2019|              1.95|
|2019|              4.95|
|2019|              5.95|
|2019|              4.95|
|2019|              5.95|
|2019|              9.95|
|2019|              9.95|
|2019|              2.95|
|2019|              5.95|
|2019|               0.8|
+----+------------------+
only showing top 20 rows



In [0]:
# groupby year to get the sum of all of the profits
result_1 = df_profits_1.groupBy("Year").agg(sum("Profits_Resturant1").alias("Total Profit Resturant 1"))

# format the Total Profit column to display as two decimal places
result_1 = result_1.withColumn("Total Profit Resturant 1", format_number("Total Profit Resturant 1", 2))
# removal off the commas
result_1 = result_1.withColumn("Total Profit Resturant 1", regexp_replace("Total Profit Resturant 1", ",", ""))
result_1.show()

+----+------------------------+
|Year|Total Profit Resturant 1|
+----+------------------------+
|2018|                58543.35|
|2015|                  262.95|
|2019|                42208.80|
|2016|                22605.85|
|2017|                47449.75|
|   0|               275446.75|
+----+------------------------+



### Resturant 2

In [0]:
# view resturant 2 
resturant_2_order.show()

+--------+----------+--------------------+--------+-------------+--------------+----+
|Order ID|Order Date|           Item Name|Quantity|Product Price|Total products|Year|
+--------+----------+--------------------+--------+-------------+--------------+----+
|   25583|2019-03-08|Tandoori Mixed Grill|       1|        11.95|            12|2019|
|   25583|2019-03-08|        Madras Sauce|       1|         3.95|            12|2019|
|   25583|2019-03-08|       Mushroom Rice|       2|         3.95|            12|2019|
|   25583|2019-03-08|         Garlic Naan|       1|         2.95|            12|2019|
|   25583|2019-03-08|             Paratha|       1|         2.95|            12|2019|
|   25583|2019-03-08|          Plain Rice|       1|         2.95|            12|2019|
|   25583|2019-03-08|         Prawn Puree|       1|         4.95|            12|2019|
|   25583|2019-03-08|       Plain Papadum|       1|          0.8|            12|2019|
|   25583|2019-03-08|       Mango Chutney|       2|   

In [0]:
# creating the profit columns by multiplying the product price with quantity
df_new_col2 = resturant_2_order.withColumn("Profits_Resturant2", col("Product Price") * col("Quantity"))
df_new_col2.show()

+--------+----------+--------------------+--------+-------------+--------------+----+------------------+
|Order ID|Order Date|           Item Name|Quantity|Product Price|Total products|Year|Profits_Resturant2|
+--------+----------+--------------------+--------+-------------+--------------+----+------------------+
|   25583|2019-03-08|Tandoori Mixed Grill|       1|        11.95|            12|2019|             11.95|
|   25583|2019-03-08|        Madras Sauce|       1|         3.95|            12|2019|              3.95|
|   25583|2019-03-08|       Mushroom Rice|       2|         3.95|            12|2019|               7.9|
|   25583|2019-03-08|         Garlic Naan|       1|         2.95|            12|2019|              2.95|
|   25583|2019-03-08|             Paratha|       1|         2.95|            12|2019|              2.95|
|   25583|2019-03-08|          Plain Rice|       1|         2.95|            12|2019|              2.95|
|   25583|2019-03-08|         Prawn Puree|       1|    

In [0]:
# creating a subdataframe by selcting which columns we want from resturant 2
df_profits_2 = df_new_col2.select("Year","Profits_Resturant2")
df_profits_2.show()

+----+------------------+
|Year|Profits_Resturant2|
+----+------------------+
|2019|             11.95|
|2019|              3.95|
|2019|               7.9|
|2019|              2.95|
|2019|              2.95|
|2019|              2.95|
|2019|              4.95|
|2019|               0.8|
|2019|               1.0|
|2019|               0.5|
|2019|               0.5|
|2019|              8.95|
|2019|             12.95|
|2019|              2.95|
|2019|              2.95|
|2019|              2.95|
|2019|              8.95|
|2019|              5.95|
|2019|              1.95|
|2019|              3.95|
+----+------------------+
only showing top 20 rows



In [0]:
# group by year and calculate the sum of profits
result_2 = df_profits_2.groupBy("Year").agg(sum("Profits_Resturant2").alias("Total Profit Resturant 2"))

# format the Total Profit column to display as two decimal places
result_2 = result_2.withColumn("Total Profit Resturant 2", format_number("Total Profit Resturant 2", 2))
result_2 = result_2.withColumn("Total Profit Resturant 2", regexp_replace("Total Profit Resturant 2", ",", ""))
result_2.show()

+----+------------------------+
|Year|Total Profit Resturant 2|
+----+------------------------+
|2018|                92784.15|
|2019|                61363.75|
|2016|                33394.10|
|2017|                81391.30|
|   0|               418165.55|
+----+------------------------+



### Combining the two profits together

In [0]:
# combining the two profits together
# join the DataFrames based on a common column
joined_profits = result_1.join(result_2, on="Year", how="outer")
joined_profits.show()

+----+------------------------+------------------------+
|Year|Total Profit Resturant 1|Total Profit Resturant 2|
+----+------------------------+------------------------+
|   0|               275446.75|               418165.55|
|2015|                  262.95|                    null|
|2016|                22605.85|                33394.10|
|2017|                47449.75|                81391.30|
|2018|                58543.35|                92784.15|
|2019|                42208.80|                61363.75|
+----+------------------------+------------------------+



In [0]:
# removing the null values and adding 0 in place of it
# define the columns to be filled with 0
columns_to_fill = ["Total Profit Resturant 2"]
# fill null values in specified columns with 0
joined_profits = joined_profits.withColumn("Total Profit Resturant 2", when(col("Total Profit Resturant 2").isNull(), 0).otherwise(col("Total Profit Resturant 2")))

# show the DataFrame with null values replaced by 0
joined_profits.show()

+----+------------------------+------------------------+
|Year|Total Profit Resturant 1|Total Profit Resturant 2|
+----+------------------------+------------------------+
|   0|               275446.75|               418165.55|
|2015|                  262.95|                       0|
|2016|                22605.85|                33394.10|
|2017|                47449.75|                81391.30|
|2018|                58543.35|                92784.15|
|2019|                42208.80|                61363.75|
+----+------------------------+------------------------+



In [0]:
# select the columns and compute their sum and make sure it is returned as 2 decimal place 
# as well to remove the commas as there was comma coming up
sum_joined_profits = joined_profits.select(
    regexp_replace(format_number(sum("Total Profit Resturant 1"), 2), ",", "").alias("Sum Resturant 1"),
    regexp_replace(format_number(sum("Total Profit Resturant 2"), 2), ",", "").alias("Sum Resturant 2")
)

sum_joined_profits.show()

+---------------+---------------+
|Sum Resturant 1|Sum Resturant 2|
+---------------+---------------+
|      446517.45|      687098.85|
+---------------+---------------+



In [0]:
# caculating the profit difference between resuturant 1 and 2
profit_difference = sum_joined_profits.withColumn("Subtraction", col("Sum Resturant 1") - col("Sum Resturant 2"))
profit_difference.show()

+---------------+---------------+-------------------+
|Sum Resturant 1|Sum Resturant 2|        Subtraction|
+---------------+---------------+-------------------+
|      446517.45|      687098.85|-240581.39999999997|
+---------------+---------------+-------------------+



### Answer
Based on the proft differnce between the resturant it shows that resturant 1 is performing worse then resturant 2 over the course of five years as the profit difference between "Sum Resturant 1" and "Sum Resturant 2" = -240581.40 (rounded up to 2 decmial place).

## Question 3: What is the the most expensive products from the two resturants? What is their least expensive product?

### Resturant 1

In [0]:
# viewing the price from resturant 1 
resturant_1_price.show()

+-------------+-------------+
|    Item Name|Product Price|
+-------------+-------------+
|   Mint Sauce|          0.5|
|  Lime Pickle|          0.5|
|Mango Chutney|          0.5|
|    Red Sauce|          0.5|
|Onion Chutney|          0.5|
|Plain Papadum|          0.8|
|Spicy Papadum|          0.8|
|      Chapati|         1.95|
|        Puree|         1.95|
|Tandoori Roti|          2.6|
|   Plain Naan|          2.6|
|       Raitha|         2.95|
|   Keema Naan|         2.95|
|Peshwari Naan|         2.95|
|   Pilau Rice|         2.95|
|  Green Salad|         2.95|
|  Bottle Coke|         2.95|
|   Plain Rice|         2.95|
|  Garlic Naan|         2.95|
|   Onion Naan|         2.95|
+-------------+-------------+
only showing top 20 rows



In [0]:
# filtering the price from most expensive price
# decided to select 12.95 - to give it more comparison of the product 
filtered_resturant_1_price = resturant_1_price.filter(col("Product Price") >= 12.95)
filtered_resturant_1_price.show()

+--------------------+-------------+
|           Item Name|Product Price|
+--------------------+-------------+
|Tandoori King Pra...|        12.95|
|Chicken Shashlick...|        12.95|
|King Prawn Hari M...|        12.95|
|Hazary Lamb Chill...|        12.95|
|Lamb Shashlick Curry|        12.95|
| King Prawn Shaslick|        12.95|
|Dhansak - King Prawn|        12.95|
|   Bengal King Prawn|        12.95|
|  Korma - King Prawn|        12.95|
|  Bhuna - King Prawn|        12.95|
|         Hazary Lamb|        12.95|
| Bengal Fish Biryani|        12.95|
|Tandoori King Pra...|        12.95|
|  King Prawn Biryani|        12.95|
|   Saag - King Prawn|        12.95|
|Dupiaza - King Prawn|        12.95|
| Pathia - King Prawn|        12.95|
|     Bengal Fry Fish|        12.95|
|    King Prawn Balti|        12.95|
|  Bengal Fish Karahi|        12.95|
+--------------------+-------------+
only showing top 20 rows



In [0]:
# sort the dataframe by price from highest with the top 10 most expensive products 
max_price_1 = filtered_resturant_1_price.orderBy(desc("Product Price")).limit(10)
max_price_1.show()

+--------------------+-------------+
|           Item Name|Product Price|
+--------------------+-------------+
| House Red wine 75cl|        17.95|
|House white wine ...|        17.95|
|Tandoori King Pra...|        12.95|
|Chicken Shashlick...|        12.95|
|King Prawn Hari M...|        12.95|
|Hazary Lamb Chill...|        12.95|
|Lamb Shashlick Curry|        12.95|
| King Prawn Shaslick|        12.95|
|Dhansak - King Prawn|        12.95|
|   Bengal King Prawn|        12.95|
+--------------------+-------------+



In [0]:
# sort the dataframe by price from lowest with the top 10 most cheapest products 
lowest_price_1 = resturant_1_price.orderBy(asc("Product Price")).limit(10)
lowest_price_1.show()

+-------------+-------------+
|    Item Name|Product Price|
+-------------+-------------+
|   Mint Sauce|          0.5|
|  Lime Pickle|          0.5|
|Mango Chutney|          0.5|
|    Red Sauce|          0.5|
|Onion Chutney|          0.5|
|Plain Papadum|          0.8|
|Spicy Papadum|          0.8|
|      Chapati|         1.95|
|        Puree|         1.95|
|Tandoori Roti|          2.6|
+-------------+-------------+



### Resturant 2

In [0]:
# viewing the price from resturant 2 
resturant_2_price.show()

+-------------+-------------+
|    Item Name|Product Price|
+-------------+-------------+
|Onion Chutney|          0.5|
|   Mint Sauce|          0.5|
|Mango Chutney|          0.5|
|    Red Sauce|          0.5|
|  Lime Pickle|          0.5|
|Plain Papadum|          0.8|
|Spicy Papadum|          0.8|
|      Chapati|         1.95|
|        Puree|         1.95|
|         Dahi|         1.95|
|         Naan|          2.5|
|Tandoori Roti|          2.5|
|   Keema Naan|         2.95|
|   Pilau Rice|         2.95|
|  Garlic Naan|         2.95|
|   Plain Rice|         2.95|
|  Bottle Coke|         2.95|
|Peshwari Naan|         2.95|
|      Paratha|         2.95|
|  Green Salad|         2.95|
+-------------+-------------+
only showing top 20 rows



In [0]:
# filtering the price from most expensive price
# decided to select 12.95 - to give it more comparison of the product 
filtered_resturant_2_price = resturant_2_price.filter(col("Product Price") >= 12.95)
filtered_resturant_2_price.show()

+--------------------+-------------+
|           Item Name|Product Price|
+--------------------+-------------+
|  King Prawn Biryani|        12.95|
|Dhansak - King Prawn|        12.95|
|    King Prawn Balti|        12.95|
|  Korma - King Prawn|        12.95|
|Tandoori King Pra...|        12.95|
| King Prawn Shaslick|        12.95|
|    King Prawn Royal|        12.95|
|Tandoori King Pra...|        12.95|
| Pathia - King Prawn|        12.95|
|   King Prawn Karahi|        12.95|
|  Bhuna - King Prawn|        12.95|
|      Jinga Chillies|        12.95|
| King Prawn Jalfrezi|        12.95|
|  Cylon - King Prawn|        12.95|
| Bombay - King Prawn|        12.95|
|Dupiaza - King Prawn|        12.95|
|Mushroom - King P...|        12.95|
|  Rogon - King Prawn|        12.95|
|   Saag - King Prawn|        12.95|
| Pathia - king-prawn|        12.95|
+--------------------+-------------+
only showing top 20 rows



In [0]:
# sort the dataframe by price from highest with the top 10 most expensive products 
max_price_2 = filtered_resturant_2_price.orderBy(desc("Product Price")).limit(10)
max_price_2.show()

+--------------------+-------------+
|           Item Name|Product Price|
+--------------------+-------------+
|House white wine ...|        17.95|
| House Red wine 75cl|        17.95|
|  King Prawn Biryani|        12.95|
|Dhansak - King Prawn|        12.95|
|    King Prawn Balti|        12.95|
|  Korma - King Prawn|        12.95|
|Tandoori King Pra...|        12.95|
| King Prawn Shaslick|        12.95|
|    King Prawn Royal|        12.95|
|Tandoori King Pra...|        12.95|
+--------------------+-------------+



In [0]:
# sort the dataframe by price from lowest with the top 10 most cheapest products 
lowest_price_2 = resturant_2_price.orderBy(asc("Product Price")).limit(10)
lowest_price_2.show()

+-------------+-------------+
|    Item Name|Product Price|
+-------------+-------------+
|Onion Chutney|          0.5|
|   Mint Sauce|          0.5|
|Mango Chutney|          0.5|
|    Red Sauce|          0.5|
|  Lime Pickle|          0.5|
|Plain Papadum|          0.8|
|Spicy Papadum|          0.8|
|      Chapati|         1.95|
|        Puree|         1.95|
|         Dahi|         1.95|
+-------------+-------------+



### Answer
From the analyses what is the most interesting similiarity between the rwo resturants is White Wine adn Red Wine are their both of their most expensie products. For resturant 1 their most expensive products three products are Dupiaza - king-prawn,Saag - king-prawn, Saag - king-prawn and resturant 2 are King Prawn Biryani, King Prawn Balti, Dhansak - King Prawn. For the cheapest products for both resturants have similiar products as it mainly cosnsits of sauces such as (mint sauces, onion chutney and mango chutney etc) as well as the papadum(lime, plain and spicy etc )

## Question 4: Which customer from the two resturants placed the most order, for all of the years?

### Resturant 1

In [0]:
# see the resturant 1 order again 
resturant_1_order.show()

+------------+----------+--------------------+--------+-------------+--------------+----+
|Order Number|Order Date|           Item Name|Quantity|Product Price|Total products|Year|
+------------+----------+--------------------+--------+-------------+--------------+----+
|       16118|2019-03-08|       Plain Papadum|       2|          0.8|             6|2019|
|       16118|2019-03-08|    King Prawn Balti|       1|        12.95|             6|2019|
|       16118|2019-03-08|         Garlic Naan|       1|         2.95|             6|2019|
|       16118|2019-03-08|       Mushroom Rice|       1|         3.95|             6|2019|
|       16118|2019-03-08| Paneer Tikka Masala|       1|         8.95|             6|2019|
|       16118|2019-03-08|       Mango Chutney|       1|          0.5|             6|2019|
|       16117|2019-03-08|          Plain Naan|       1|          2.6|             7|2019|
|       16117|2019-03-08|       Mushroom Rice|       1|         3.95|             7|2019|
|       16

In [0]:
# checking for the resturant data types
resturant_1_order.dtypes

Out[282]: [('Order Number', 'int'),
 ('Order Date', 'date'),
 ('Item Name', 'string'),
 ('Quantity', 'int'),
 ('Product Price', 'double'),
 ('Total products', 'int'),
 ('Year', 'int')]

In [0]:
# creating a subset dataframe 
# filter > 2016 - makes it easier for comparison when comparing with resturant 2
df_order_1 = resturant_1_order.select(col("Order Number"), col("Quantity"), col("Year")).filter(col("Year") >= 2016)
df_order_1.show()

+------------+--------+----+
|Order Number|Quantity|Year|
+------------+--------+----+
|       16118|       2|2019|
|       16118|       1|2019|
|       16118|       1|2019|
|       16118|       1|2019|
|       16118|       1|2019|
|       16118|       1|2019|
|       16117|       1|2019|
|       16117|       1|2019|
|       16117|       1|2019|
|       16117|       1|2019|
|       16117|       1|2019|
|       16117|       1|2019|
|       16117|       1|2019|
|       16116|       1|2019|
|       16116|       1|2019|
|       16116|       1|2019|
|       16116|       1|2019|
|       16116|       1|2019|
|       16115|       1|2019|
|       16115|       1|2019|
+------------+--------+----+
only showing top 20 rows



In [0]:
# group by the Order Number to count the number of quantities
grouped_order_1 = df_order_1.groupBy("Order Number").agg(count("Quantity").alias("Quantity Count"))
grouped_order_1.show()

+------------+--------------+
|Order Number|Quantity Count|
+------------+--------------+
|        9465|             6|
|        9427|             1|
|        8086|             7|
|        7754|            15|
|        7253|             4|
|        6620|             8|
|        2659|            13|
|        6654|             6|
|        7340|            12|
|        6357|             4|
|       13623|             4|
|       12799|             9|
|       11858|             5|
|       11458|             6|
|       10623|             7|
|       10206|             7|
|        5300|             3|
|        3918|             4|
|       13289|             4|
|       13285|             5|
+------------+--------------+
only showing top 20 rows



In [0]:
# sort the grouped DataFrame by "Quantity Count" column in descending order and getting the top 5
sorted_order_1 = grouped_order_1.orderBy(desc("Quantity Count")).limit(5)
sorted_order_1.show()

+------------+--------------+
|Order Number|Quantity Count|
+------------+--------------+
|        9412|            60|
|        9414|            58|
|        9411|            45|
|        9413|            27|
|        9410|            26|
+------------+--------------+



### Resturant 2

In [0]:
# see the resturant 2 order again 
resturant_2_order.show()

+--------+----------+--------------------+--------+-------------+--------------+----+
|Order ID|Order Date|           Item Name|Quantity|Product Price|Total products|Year|
+--------+----------+--------------------+--------+-------------+--------------+----+
|   25583|2019-03-08|Tandoori Mixed Grill|       1|        11.95|            12|2019|
|   25583|2019-03-08|        Madras Sauce|       1|         3.95|            12|2019|
|   25583|2019-03-08|       Mushroom Rice|       2|         3.95|            12|2019|
|   25583|2019-03-08|         Garlic Naan|       1|         2.95|            12|2019|
|   25583|2019-03-08|             Paratha|       1|         2.95|            12|2019|
|   25583|2019-03-08|          Plain Rice|       1|         2.95|            12|2019|
|   25583|2019-03-08|         Prawn Puree|       1|         4.95|            12|2019|
|   25583|2019-03-08|       Plain Papadum|       1|          0.8|            12|2019|
|   25583|2019-03-08|       Mango Chutney|       2|   

In [0]:
# creating a subset dataframe 
df_order_2 = resturant_2_order.select(col("Order ID"), col("Quantity"), col("Year"))
df_order_2.show()

+--------+--------+----+
|Order ID|Quantity|Year|
+--------+--------+----+
|   25583|       1|2019|
|   25583|       1|2019|
|   25583|       2|2019|
|   25583|       1|2019|
|   25583|       1|2019|
|   25583|       1|2019|
|   25583|       1|2019|
|   25583|       1|2019|
|   25583|       2|2019|
|   25583|       1|2019|
|   25583|       1|2019|
|   25583|       1|2019|
|   25582|       1|2019|
|   25582|       1|2019|
|   25582|       1|2019|
|   25582|       1|2019|
|   25582|       1|2019|
|   25581|       1|2019|
|   25581|       1|2019|
|   25581|       1|2019|
+--------+--------+----+
only showing top 20 rows



In [0]:
# group by the Order Number to count the number of quantities
grouped_order_2 = df_order_2.groupBy("Order ID").agg(count("Quantity").alias("Quantity Count"))
grouped_order_2.show()

+--------+--------------+
|Order ID|Quantity Count|
+--------+--------------+
|   25517|             4|
|   24663|             5|
|   24347|            10|
|   24171|             3|
|   21700|             7|
|   21220|             6|
|   20924|             5|
|   20497|            11|
|   20382|             8|
|   19553|             7|
|   19204|             6|
|   18944|             9|
|   18498|             6|
|   12046|            11|
|   11858|             8|
|   11033|             5|
|   10623|             4|
|   10362|             6|
|   10206|             9|
|    9900|            11|
+--------+--------------+
only showing top 20 rows



In [0]:
# sort the grouped DataFrame by "Quantity Count" column in descending order and getting the top 5
sorted_order_2 = grouped_order_2.orderBy(desc("Quantity Count")).limit(5)
sorted_order_2.show()

+--------+--------------+
|Order ID|Quantity Count|
+--------+--------------+
|    7952|            29|
|   19962|            25|
|   13246|            24|
|   19569|            24|
|   21228|            24|
+--------+--------------+



### Getting the quanity of Order Id for both resturants
Makes it easier for comparisn when analysing

In [0]:
# show resturant 1
sorted_order_1.show()

+------------+--------------+
|Order Number|Quantity Count|
+------------+--------------+
|        9412|            60|
|        9414|            58|
|        9411|            45|
|        9413|            27|
|        9410|            26|
+------------+--------------+



In [0]:
# show resturant 2
sorted_order_2.show()

+--------+--------------+
|Order ID|Quantity Count|
+--------+--------------+
|    7952|            29|
|   19962|            25|
|   13246|            24|
|   19569|            24|
|   21228|            24|
+--------+--------------+



### Answer
From the analyses from above it has shows that the top 5 Order ID or Order Number and the number of times that customer had ordered from resturant 1 during the course of 2016-2019 are 9412:60,9414:58,9411:25,9413:27,9410:26 and where for resturant 2 are 7952:29,19962:25,13246:24,19569:24,2122:24 from 2016-2019. This will be further dicscussed in the conclusion section.

# Conclusion

In conclusion, based on the the data exploration have has discovered the answers to answer the four questions below:

Question 1: Which resturant has the most sales by delivery and by how much?
- Answer: The most sales which the resturant has is Resturant 2, as resturant 2 shows significantly higher sales in product for example for year 2016 it shows that resturant 2 sold 1968 more products the resturant 1 which is in the postive numbers and so forth with the following year. However there is -49 between the resturanr_2 and resturant_2 count and the possible reason the resturant 2 has not opened in 2015.

Question 2: Which resturants produced the highest profits for during the course of 5 years?
- Answer: Based on the proft differnce between the resturant it shows that resturant 1 is performing worse then resturant 2 over the course of five years as the profit difference between "Sum Resturant 1" and "Sum Resturant 2" = -240581.40 (rounded up to 2 decmial place).


Question 3: What is the the most expensive products from the two resturants? What is their least expensive product?
- Answer: From the analyses what is the most interesting similiarity between the rwo resturants is White Wine adn Red Wine are their both of their most expensie products. For resturant 1 their most expensive products three products are Dupiaza - king-prawn,Saag - king-prawn, Saag - king-prawn and resturant 2 are King Prawn Biryani, King Prawn Balti, Dhansak - King Prawn. For the cheapest products for both resturants have similiar products as it mainly cosnsits of sauces such as (mint sauces, onion chutney and mango chutney etc) as well as the papadum(lime, plain and spicy etc ).

Question 4: Which customer from the two resturants placed the most order, for all of the years?
- AnswerFrom the analyses from above it has shows that the top 5 Order ID or Order Number and the number of times that customer had ordered from resturant 1 during the course of 2016-2019 are 9412:60,9414:58,9411:25,9413:27,9410:26 and where for resturant 2 are 7952:29,19962:25,13246:24,19569:24,2122:24 from 2016-2019. This will be further dicscussed in the conclusion section.

In summary, based on the four answers it has been given it is evidently shown that resturant 2 is doing better then resturant 1, even though it has opened a year later, based on the product sold count and the profit difference. The possible reason why resturant 2 is doing better is that, it miight be in a good location where potentital customers can visbily see the resturant or it is the Indian Takeway Company might heavily invested in markerting for resturant 2. Based on these two reason, it offers a potential good explanation why resturant 2 is doing better instead of who makes the most profits as based on the number of prices they have chared. As based from answer from Question 3 it shows that two resturants are eqaully priced for the similar prodcuts.

However, one thing that resturant 1 does better in compairson to resturant 2 is the amount of returning customers they have between the 5 years the most order that a customer placed in ID Number 9412 with an order of 60, where's resturant 2 highest order id ID Number 7952: 29. Which 31 differnce between these two and the rest of the top 5 customers follow the relative same cusomters.

Therefore to end this conclusion offer, resturant 2 does well in making profits howver resturant 1 has more loyal customer base based on the number or order from the same customer from 2016-2019.